# Projeto 1 - Ciência dos Dados

Nome: GUSTAVO CAMARGO

Nome: TIAGO SEIXAS

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
filename = 'Apple.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Apple.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,paaaaaaara com isso apple https://t.co/cetaiesil8,0
1,objetivo: falar sobre os novos produtos da app...,0
2,gente a apple n me da 1 min d paz eu ein,1
3,@prk_bora conte comigo,0
4,rt @sadbeautifulmel: stream party começando em...,0


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,aprovamos celulares de todas as marcas: samsun...,0
1,"eu sempre defendo a @samsungbrasil, mas fica d...",1
2,dêem rt/fav aqui!!\nbenfica jorge jesus paok #...,0
3,apple pelo amor de deus para de lança atualiza...,1
4,@xboxnewsreddit @siralysson olha a microsoft a...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

**R:** Nosso produto é a marca de tecnologia Apple. Recentemente essa empresa realizou um evento para apresentar suas novas novidades. Portanto, consideramos que tweets que falassem de produtos, eventos e qualidades gerais da apple seriam considerados relevantes. Por outro lado, tweets que não tivessem muita relação com o que foi dito acima, seriam considerados irrelevantes para a análise.     

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### > Funções que serão úteis para o desenvolvimento do projeto:

In [5]:
def cleanup(text):
    punctuation = '[!-.:?;\n$%#@)(\/“_ショッポ]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def separa(train):
    #Transformando a Serie para uma lista
    lista_train = train.Treinamento.tolist()
    
    #Transforma a lista em uma string
    teste = ''.join([str(frase) for frase in lista_train])
    
    #Separa os items da string e forma uma lista 
    teste_separado = teste.lower().split()
    
    #Volta para uma Series
    teste_serie = pd.Series(teste_separado)
    return teste_serie

#### > Iniciando processo de limpeza:

In [6]:
train.Treinamento = train.Treinamento.apply(cleanup)
train.head(5)

,Treinamento,Relevancia
0,paaaaaaara com isso apple httpstcocetaiesil8,0
1,objetivo falar sobre os novos produtos da appl...,0
2,gente a apple n me da 1 min d paz eu ein,1
3,prkbora conte comigo,0
4,rt sadbeautifulmel stream party começando em 2...,0


In [7]:
test.Teste= test.Teste.apply(cleanup)
test.head(5)

,Teste,Relevancia
0,aprovamos celulares de todas as marcas samsung...,0
1,eu sempre defendo a samsungbrasil mas fica dif...,1
2,dêem rtfav aquibenfica jorge jesus paok applee...,0
3,apple pelo amor de deus para de lança atualiza...,1
4,xboxnewsreddit siralysson olha a microsoft ata...,0


#### > Repare que nossa limpeza não removeu os emojis, como foi pedido:


In [8]:
test.iloc[[29],:]

,Teste,Relevancia
29,[📸] vou jogar umas verdades no colo de vocêsam...,0


#### > Categorizando as colunas, a fim de facilitar análise:

In [9]:
train.loc[:,'Relevancia'] = train['Relevancia'].astype('category')
train.Relevancia.cat.categories = ['Irrelevante', 'Relevante']

train.head(5)

,Treinamento,Relevancia
0,paaaaaaara com isso apple httpstcocetaiesil8,Irrelevante
1,objetivo falar sobre os novos produtos da appl...,Irrelevante
2,gente a apple n me da 1 min d paz eu ein,Relevante
3,prkbora conte comigo,Irrelevante
4,rt sadbeautifulmel stream party começando em 2...,Irrelevante


In [10]:
test.loc[:,'Relevancia'] = test['Relevancia'].astype('category')
test.Relevancia.cat.categories = ['Irrelevante', 'Relevante']

test.head(5)

,Teste,Relevancia
0,aprovamos celulares de todas as marcas samsung...,Irrelevante
1,eu sempre defendo a samsungbrasil mas fica dif...,Relevante
2,dêem rtfav aquibenfica jorge jesus paok applee...,Irrelevante
3,apple pelo amor de deus para de lança atualiza...,Relevante
4,xboxnewsreddit siralysson olha a microsoft ata...,Irrelevante


#### > Iniciando o treinamento do classificador:

In [11]:
#Achando as frequências de cada palavra relevante: 
filtro_relevante = train.loc[train.Relevancia == 'Relevante',:]
relevante = separa(filtro_relevante) #Isso é uma series

count_rel = relevante.value_counts() 
n_rel = count_rel.sum()

count_rel.head(5)

apple    159
o        104
a         97
que       80
de        69
dtype: int64

In [12]:
#Achando as frequências de cada palavra irrelevante: 
filtro_irrelevante = train.loc[train.Relevancia == 'Irrelevante',:]
irrelevante = separa(filtro_irrelevante) #Isso é uma series

count_irrel = irrelevante.value_counts()
n_irrel = count_rel.sum()

count_irrel.head(5)

apple    111
e         50
de        47
a         44
da        41
dtype: int64

In [13]:
#Achando probabilidades a priori: 

priori = train.Relevancia.value_counts(True, sort = False)
p_irrelevante_log = np.log(priori[0])
p_relevante_log = np.log(priori[1])

print(p_irrelevante_log, p_relevante_log)

-0.7985076962177716 -0.5978370007556204


In [14]:
#Achando probabilidades condicionais:

#Smoothing
alfa = 1

#Chute de quantas palavras tem no vocabulário
v = 1000000

#Condições iniciais 
log_probP_dadoRelevante=1
log_probP_dadoIrrelevante=1
log_prob_desconhecida = 1

log_prob_desconhecida = np.log(alfa/(n_rel + alfa*v))
log_prob_desconhecida_ir = np.log(alfa/(n_irrel +  alfa*v))

#Criando as listas
listaProbs_r = []
listaProbs_irr = []

i=0

while i<len(train):
    for palavra in train.iloc[i,0].split():
    
        #Se a palavra estiver no relevante:
        if palavra in relevante.tolist():
            log_probP_dadoRelevante += np.log((count_rel[palavra] + alfa)/(n_rel + alfa*v))    
        else:
            log_probP_dadoRelevante += log_prob_desconhecida
            
            
    #Se a palavra estiver no irrelevante:
        if palavra in irrelevante.tolist():
            log_probP_dadoIrrelevante += np.log((count_irrel[palavra] + alfa)/(n_irrel + alfa*v))
        
        else:
            log_probP_dadoIrrelevante += log_prob_desconhecida_ir 
            
        
    listaProbs_r.append(log_probP_dadoRelevante)
    listaProbs_irr.append(log_probP_dadoIrrelevante)
    
    log_probP_dadoRelevante = 0
    log_probP_dadoIrrelevante = 0
    
    i+=1
    

print(len(listaProbs_r))
print(len(listaProbs_irr))


300
300


In [15]:
lista_Relevante_dadoP = []
lista_Irrelevante_dadoP = []

for i in range(0,300):
    
    lista_Relevante_dadoP.append(listaProbs_r[i] + p_relevante_log)
    lista_Irrelevante_dadoP.append(listaProbs_irr[i] + p_irrelevante_log)
    
print(len(lista_Relevante_dadoP))
print(len(lista_Irrelevante_dadoP))

300
300


In [16]:
train['RdadoP'] = np.array(lista_Relevante_dadoP)
train['Irdadop'] = np.array(lista_Irrelevante_dadoP)

In [17]:
train['Naive_Bayes'] = np.where(train['RdadoP'] > train['Irdadop'],'Relevante','Irrelevante')
train.head(5)

,Treinamento,Relevancia,RdadoP,Irdadop,Naive_Bayes
0,paaaaaaara com isso apple httpstcocetaiesil8,Irrelevante,-58.240129,-59.162118,Relevante
1,objetivo falar sobre os novos produtos da appl...,Irrelevante,-209.233488,-201.908844,Irrelevante
2,gente a apple n me da 1 min d paz eu ein,Relevante,-139.515369,-148.407621,Relevante
3,prkbora conte comigo,Irrelevante,-42.053484,-41.561007,Irrelevante
4,rt sadbeautifulmel stream party começando em 2...,Irrelevante,-260.908648,-245.657587,Irrelevante


In [24]:
tabela = pd.crosstab(train.Naive_Bayes, train.Relevancia, normalize=True)
tabela

Relevancia,Irrelevante,Relevante
Naive_Bayes,,
Irrelevante,0.27,0.00
Relevante,0.18,0.55


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [30]:
#Zerando as listas e reiniciando processo de cálculo de probabilidades

del listaProbs_r[:]
del listaProbs_irr[:]

listaProbs_r = []
listaProbs_irr = []

i=0

while i<len(test):
    for palavra in test.iloc[i,0].split():
    
        #Se a palavra estiver no relevante:
        if palavra in relevante.tolist():
            log_probP_dadoRelevante += np.log((count_rel[palavra] + alfa)/(n_rel + alfa*v))    
        else:
            log_probP_dadoRelevante += log_prob_desconhecida
            
            
    #Se a palavra estiver no irrelevante:
        if palavra in irrelevante.tolist():
            log_probP_dadoIrrelevante += np.log((count_irrel[palavra] + alfa)/(n_irrel + alfa*v))
        
        else:
            log_probP_dadoIrrelevante += log_prob_desconhecida_ir 
            
        
    listaProbs_r.append(log_probP_dadoRelevante)
    listaProbs_irr.append(log_probP_dadoIrrelevante)
    
    log_probP_dadoRelevante = 0
    log_probP_dadoIrrelevante = 0
    
    i+=1


200


In [28]:
#Recriando as listas 
del lista_Relevante_dadoP[:]
del lista_Irrelevante_dadoP[:]

lista_Relevante_dadoP = []
lista_Irrelevante_dadoP = []

#Calcula probabilidades finais
for i in range(0,200):
    
    lista_Relevante_dadoP.append(listaProbs_r[i] + p_relevante_log)
    lista_Irrelevante_dadoP.append(listaProbs_irr[i] + p_irrelevante_log)
    
#Adicionando ao DataFrame
test['RdadoP'] = np.array(lista_Relevante_dadoP)
test['Irdadop'] = np.array(lista_Irrelevante_dadoP)

#Classificando
test['Naive_Bayes'] = np.where(test['RdadoP'] > test['Irdadop'],'Relevante','Irrelevante')
test.head(5)

,Teste,Relevancia,RdadoP,Irdadop,Naive_Bayes
0,aprovamos celulares de todas as marcas samsung...,Irrelevante,-248.648914,-255.603606,Relevante
1,eu sempre defendo a samsungbrasil mas fica dif...,Relevante,-275.921188,-282.063292,Relevante
2,dêem rtfav aquibenfica jorge jesus paok applee...,Irrelevante,-349.260424,-306.576285,Irrelevante
3,apple pelo amor de deus para de lança atualiza...,Relevante,-149.327020,-158.691442,Relevante
4,xboxnewsreddit siralysson olha a microsoft ata...,Irrelevante,-135.178713,-139.777575,Relevante


In [29]:
tabela_final = pd.crosstab(test.Naive_Bayes, test.Relevancia, normalize=True)
tabela_final

Relevancia,Irrelevante,Relevante
Naive_Bayes,,
Irrelevante,0.07,0.01
Relevante,0.41,0.51


-  Porcentagem de verdadeiros positivos: 51%
-  Porcentagem de falsos positivos: 41%
-  Porcentagem de verdadeiros negativos: 7%
-  Porcentagem de falsos negativos: 1%

Pode-se ver que o nosso classificador Naïve Bayes apresentou 58% de veracidade.

___
### Concluindo

#### > Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)

- Uma forma de melhorar um classificador do tipo Naïve Bayes seria não assumir que os elementos existem individualmente, usando ao invés disso os elementos como sendo conjuntos, usando de uma correlação entre os elementos e uma determinada classe, usando disso para determinar a importância dos mesmos.

- Uma maneira de fazer melhorias reais seria remover Stop words, ou seja, palavras que não agregam valor algum para a análise de dados (repare que isso abrange um conjunto maior de palavras do que a limpeza que nós indicamos no item de novas limpezas). Todavia, essa seleção deve ser feita com MUITO cuidado, visto que da mesma forma que pode beneficiar o modelo, também pode prejudica-lo!

Fontes: https://www.researchgate.net/publication/262394061_Improving_Naive_Bayes_classifier_using_conditional_probabilities e
https://stats.stackexchange.com/questions/280441/stopword-removal-suprisingly-decreases-accuracy-of-naive-bayes-model

#### > Propor diferentes cenários para Naïve Bayes fora do contexto do projeto

- O modelo Naïve Bayes pode ser usado juntamente com outros algoritmos para criar sistemas de recomendação, como o usado pela Netflix para recomendar filmes.

- Outras aplicações, seria checar se um e-mail é um spam ou não (algo muito utilizado em jogos on-line. Poderia também ser utilizado para classificar se um artigo é de política, futebol, economia, dentre muitos outros assuntos.


Fontes: https://towardsdatascience.com/a-mathematical-explanation-of-naive-bayes-in-5-minutes-44adebcdb5f8 e https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used

#### > Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação

- Para melhorar a informação que se obtém com o modelo Naïve Bayes, pode-se criar uma nova função para excluir certas palavras, como por exemplo palavrões, que não agregam em nada à pesquisa.

- Outra limpeza seria remover palavras que sejam curtas demais e que não agregam valor, como por exemplo "a" , "b" , "c" . Esses tipos de palavras, artigos e frases não são muito úteis para a análise do dado. 

Fonte: https://www.codementor.io/@ilyaas97/data-cleaning-in-python-basics-using-pandas-yw18j94yg

#### > Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento

- Caso seja usado o proprio classificador para gerar mais amostras de treinamento, o mesmo não conseguirá expandir seu vocabulário e será incapaz de se tornar mais verossímio e de poder ser usado em mais situações.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**